<a href="https://colab.research.google.com/github/MoeinDSP/NLP_Projects/blob/main/Word%20Embeddings/05_Word_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word Embeddings

In this session we will make use of Word2Vec to learn distributed representations of words based on the context within which they appear in sentences.

**Optional for Colab users**

Before starting, we can set up the connection with the Google Dive storage, to keep there our documents.
Just execute the following passages:

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Make sure that the variable path contains the correct sequence of folders separate by a `'/'` to get to your lecture files

In [ ]:
import os

path = '/content/drive/MyDrive/Practical_05__Word-Embeddings'

os.chdir(f'{path}')
os.getcwd()

'/content/drive/MyDrive/Practical_05__Word-Embeddings'

## Learning word embeddings

We'll first try to learn word embeddings using the Word2Vec implementation provided by the gensim package.
- Information on gensim's word2vec implementation is available here: https://radimrehurek.com/gensim/models/word2vec.html
- First install the package using pip:

In [ ]:
!pip install --upgrade gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 17.6 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


We will build a Word2Vec embedding using the 20 Newsgroup data:

In [ ]:
from sklearn.datasets import fetch_20newsgroups

dataset_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
dataset_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

In order to train Word2Vec on the data we first need to convert it to the right format.
- For training Word2Vec, it is usual to **separate data into individual sentences** and then tokenize those sentences separately
- So let's use regular expressions to remove the end-of-line characters from each document and then split the documents into sentences using a regular expression that looks for question marks, exclamation marks, and periods, followed by a space:

In [ ]:
import re

# remove newline characters
docs = [re.sub('\n', ' ', doc) for doc in dataset_train.data]
# remove email addresses
docs = [re.sub('[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', '', doc) for doc in docs]
# split sentences
sentences = [re.split('[?!.]\s', doc) for doc in docs]
sentences[:3]

[['I was wondering if anyone out there could enlighten me on this car I saw the other day',
  'It was a 2-door sports car, looked to be from the late 60s/ early 70s',
  'It was called a Bricklin',
  'The doors were really small',
  'In addition, the front bumper was separate from the rest of the body',
  'This is  all I know',
  'If anyone can tellme a model name, engine specs, years of production, where this car is made, history, or whatever info you have on this funky looking car, please e-mail.'],
 ['A fair number of brave souls who upgraded their SI clock oscillator have shared their experiences for this poll',
  'Please send a brief message detailing your experiences with the procedure',
  'Top speed attained, CPU rated speed, add on cards and adapters, heat sinks, hour of usage per day, floppy disk functionality with 800 and 1.4 m floppies are especially requested',
  " I will be summarizing in the next two days, so please add to the network knowledge base if you have done the cl

Above, we've printed out the first two documents, which have been split into arrays of sentences.
- We'll need to flatten the structure into one big array of sentences (remove the distinction between documents) before providing it to Word2Vec.
- We can do that using the flatten command from the pandas library:

In [ ]:
from pandas.core.common import flatten

sentences = list(flatten(sentences))
sentences[:20]

['I was wondering if anyone out there could enlighten me on this car I saw the other day',
 'It was a 2-door sports car, looked to be from the late 60s/ early 70s',
 'It was called a Bricklin',
 'The doors were really small',
 'In addition, the front bumper was separate from the rest of the body',
 'This is  all I know',
 'If anyone can tellme a model name, engine specs, years of production, where this car is made, history, or whatever info you have on this funky looking car, please e-mail.',
 'A fair number of brave souls who upgraded their SI clock oscillator have shared their experiences for this poll',
 'Please send a brief message detailing your experiences with the procedure',
 'Top speed attained, CPU rated speed, add on cards and adapters, heat sinks, hour of usage per day, floppy disk functionality with 800 and 1.4 m floppies are especially requested',
 " I will be summarizing in the next two days, so please add to the network knowledge base if you have done the clock upgrade 

Now we can proceed to do some cleaning of the data:
- remove non-letter characters from each sentence
- lowercase
- tokenize the sentences based on whitespace
- remove any sentence with length less than 2 since it won't be useful for training Word2Vec.

In [ ]:
tokenized_sentences = [re.sub('\W', ' ', sentence).lower().split() for sentence in sentences]
# remove sentences that are only 1 word long
tokenized_sentences = [sentence for sentence in tokenized_sentences if len(sentence) > 1]

for sentence in tokenized_sentences[:10]:
    print(sentence)

['i', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'i', 'saw', 'the', 'other', 'day']
['it', 'was', 'a', '2', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', '60s', 'early', '70s']
['it', 'was', 'called', 'a', 'bricklin']
['the', 'doors', 'were', 'really', 'small']
['in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body']
['this', 'is', 'all', 'i', 'know']
['if', 'anyone', 'can', 'tellme', 'a', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'e', 'mail']
['a', 'fair', 'number', 'of', 'brave', 'souls', 'who', 'upgraded', 'their', 'si', 'clock', 'oscillator', 'have', 'shared', 'their', 'experiences', 'for', 'this', 'poll']
['please', 'send', 'a', 'brief', 'message', 'detailing', 'your', 'experiences', 'with

Finally we have the data in the right format for training Word2Vec, so we can provide it to the model. For parameters, we set:
- the embedding size to be 30,
- the minimum count for any vocabulary term to be 5
- the size of the context window to 10.

In [ ]:
from gensim.models.word2vec import Word2Vec

model = Word2Vec(tokenized_sentences, vector_size=30, min_count=5, window=10)

Let's see how big the vocabulary is that Word2Vec ended up using, i.e. how many word vectors did it learn:

In [12]:
len(model.wv)

22069

## Inspecting embeddings and finding similar words

Now that we have a word2vec model trained, what can we do with it?
- Let's print out one of the vectors to see what it looks like:

In [13]:
term = 'car'
model.wv[term]

array([-0.7559625 ,  0.6472946 ,  2.1280231 , -2.4135783 ,  0.18323535,
       -0.01846329,  0.1828807 , -1.0578258 , -0.4933687 ,  0.16063736,
        1.9403836 ,  1.1408741 ,  2.1406827 ,  1.3721251 ,  1.1260915 ,
       -0.26965767,  3.4714174 , -0.2595292 ,  1.9821072 ,  0.6344573 ,
       -2.9201436 , -0.8415183 , -1.1564403 ,  1.0165981 , -2.1505692 ,
       -0.9173762 ,  2.6843123 , -0.4237848 ,  4.656776  ,  1.2577894 ],
      dtype=float32)

Unlike LDA's document topic vector, this one has both positive and negative values.

We could now use the model to compute similarities between terms based on their cosine distance in the embedding space.
- Try modifying the terms below to see what their closest neighbouring terms are:
  - Some good terms to try are: hockey, mouse, god, microsoft, clinton, bush, etc.
  - (If yuo see an error, that is because the term you chose is not in the dictionary.)

In [ ]:
# term = 'gun'
# term = 'microsoft'
# term ='clinton'

model.wv.most_similar(term)

Do the similar terms agree with your intuition?

## Visualising the embedding vectors using t-SNE

We'll now visualise some of the word vectors in a 3 dimensional space using t-SNE.

The vocabulary of word vectors is quite large (around 25,000). Giving them all to t-SNE will cause it to take far too long to converge.
- So let's first choose a random subset of 500 terms to show:

In [ ]:
import random

sample = random.sample(list(model.wv.key_to_index), 500)
print(sample)

Now we'll get the word vectors for the sampled terms:

In [ ]:
word_vectors = model.wv[sample]
word_vectors

And we'll provide the vectors to TSNE to fit a model and transform the data to 3 dimensions:

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=3, n_iter=2000)
tsne_embedding = tsne.fit_transform(word_vectors)

Now transform the data into 3 columns (for x, y, and z):

In [ ]:
import numpy as np

x, y, z = np.transpose(tsne_embedding)

And generate the 3d plot:

In [ ]:
!pip install plotly

In [ ]:
import plotly.express as px

fig = px.scatter_3d(x=x, y=y, z=z)
fig.update_traces(marker=dict(size=3,line=dict(width=2)))
fig.show()

Well that's a not a particularly interesting 3d plot!
- How about we label some of the points on the graph to see what words they correspond to:

In [ ]:
fig = px.scatter_3d(x=x[:200],y=y[:200],z=z[:200],text=sample[:200])
fig.update_traces(marker=dict(size=3,line=dict(width=2)),textfont_size=10)
fig.show()

Let's extend the random set of terms with a set of colours to see if they cluster:

In [ ]:
# Add some specific terms to sample:
colours = ['red','green','blue','orange','yellow','purple','pink','cream','brown','black','white','gray']

word_vectors = model.wv[colours+sample]

tsne = TSNE(n_components=3)
tsne_embedding = tsne.fit_transform(word_vectors)

x, y, z = np.transpose(tsne_embedding)

In [ ]:
import plotly.express as px

r = (-200,200)
fig = px.scatter_3d(x=x, y=y, z=z, range_x=r, range_y=r, range_z=r, text=colours + [None] * 500)
fig.update_traces(marker=dict(size=3,line=dict(width=2)),textfont_size=10)
fig.show()

Note: t-SNE is a stochastic algorithm, so run it a couple of times to see how the visualisation changes.

Have a play around with the visualisation to see whether other sets of terms cluster together.

## Loading Pre-trained Embedding

Instead of learning your own word embeddings from a dataset, it is common to download and make use of pre-trained embeddings that have been learnt on very large corpora.
- See this list of models available in gensim: https://github.com/RaRe-Technologies/gensim-data
- Note how big some of the models are. A 300 dimensional Word2Vec model trained on Google News (word2vec-google-news-300) is almost 1.7GB!

Let's downlaod a couple of the smaller models and have a look at them:
- We'll download two GloVe (similar to Word2Vec) based models
- each has dimension of 50, but the first has been trained on data from Twitter and the second on data from Wikipedia

In [ ]:
import gensim.downloader as api

model_twitter = api.load("glove-twitter-50")
model_wiki = api.load("glove-wiki-gigaword-50")

How big are their vocabularies?

In [ ]:
print(f"Vocabulary size twitter model:   {len(model_twitter)}")
print(f"Vocabulary size wikipedia model: {len(model_wiki)}")

Woah, they are big vocabularies!
- The twitter one has over a million tokens!

Let's have a look at the most similar terms to the word 'puppy' in Twitter embedding:

In [ ]:
term = 'puppy'
print(f'Twitter embedding, most similar words to: {term}')
model_twitter.most_similar(term)

And in the Wikipedia embedding:

In [ ]:
print(f'Wikipedia embedding, most similar words to: {term}')
model_wiki.most_similar(term)

Which embedding is better?

Try some other words:

In [ ]:
term = 'apple'
# term = 'politecnico'
print(f'Twitter embedding:   {model_twitter.most_similar(term)}')
print(f'Wikipedia embedding: {model_wiki.most_similar(term)}')

Embedding spaces have interesting geometric properties, where translation between different word vectors caries semantic meaning.
- Let's try running the famous analogy: king + (woman - man) = ?
- To do that, first generate the resulting word embedding:

In [ ]:
vec = model_wiki.get_vector('king') + (model_wiki.get_vector('woman') - model_wiki.get_vector('man'))
vec

- Then look for the most similar vectors to it:

In [ ]:
model_wiki.similar_by_vector(vec)

Did it work?
- Well, yes if you remove 'king' as an option, then it did!
- Note that the most similar vector to the word 'king' was not 'queen' but 'prince', so the analogy is indeed changing the order ...

In [ ]:
model_wiki.most_similar('king')

Try it again but with the Twitter based model

In [ ]:
model = model_twitter
vec = model.get_vector('king') + (model.get_vector('woman') - model.get_vector('man'))
model.similar_by_vector(vec)

Doesn't seem that the Twitter model is as good on that analogy.
- If you think about the average quality of text on Twitter versus Wikipedia, then that's probably not suprising!

Try some other analogies:
- What happens to Rome if we subtract Italy and add France?

In [ ]:
A = 'rome'
B = 'italy'
C = 'france'
# C = 'indonesia'
model = model_wiki
vec = model.get_vector(A) - model.get_vector(B) + model.get_vector(C)
model.similar_by_vector(vec)

Now, **isn't that cool?** ;-)
- Try some other countries ...

Remember, we are only using the 'small' embeddings of 50 dimensions
- you can download and try the bigger ones, some of them with 300 dimensions


## Classifying Tweets with word embeddings

Let's try to use the word embeddings as features for a text classifier.
- In particular, we'll try the tweet sentiment analysis task from the second session.
- First download the dataset:

In [ ]:
import nltk

nltk.download('twitter_samples')

Then prepare the dataset:
- load the positive and negative tweets
- remove emoticons from them
- and merge them into a single dataset with class labels

In [ ]:
from nltk.corpus import twitter_samples

positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')

import re

emoticon_regex = '(\:\w+\:|\<[\/\\]?3|[\(\)\\\D|\*\$][\-\^]?[\:\;\=]|[\:\;\=B8][\-\^]?[3DOPp\@\$\*\\\)\(\/\|])(?=\s|[\!\.\?]|$)'
positive_tweets_noemoticons = [re.sub(emoticon_regex,'',tweet) for tweet in positive_tweets]
negative_tweets_noemoticons = [re.sub(emoticon_regex,'',tweet) for tweet in negative_tweets]

tweets_x = positive_tweets_noemoticons + negative_tweets_noemoticons
tweets_y = ['positive']*len(positive_tweets) + ['negative']*len(negative_tweets)

Divide the data into train, validation and test sets:

In [ ]:
from sklearn.model_selection import train_test_split

temp_x, test_x, temp_y, test_y = train_test_split(tweets_x, tweets_y, test_size=0.2)
train_x, valid_x, train_y, valid_y = train_test_split(temp_x, temp_y, test_size=0.2)

Now we need to convert the tweets to an embedding representation.
- We can do that by computing the sum (or average) of the embedding vectors of the words in the tweet.
- To work out how to do that, let's have a look at the first tweet in the dataset, converting it to lowercase and tokenise it on whitespace:

In [ ]:
import string
import re
regex = '[' + string.punctuation + ']'

print('tweet: \'' + train_x[10] + '\'')
tokens = re.sub(regex, '', train_x[10].lower()).split()
print('tokens: ', tokens)

- We can get the embedding vectors for the tokens present in the embedding vocabulary as follows:

In [ ]:
model = model_twitter
embeddings = [model.get_vector(token) for token in tokens if token in model]
embeddings

- the sum of the embeddings in the tweet is:

In [ ]:
sum(embeddings)

- and the average of the embeddings is just:

In [ ]:
import numpy as np

np.mean(embeddings, axis=0)

We will need to perform this vector transformation for all tweets in the training and also the test sets
- We can define a procedure to do that for us

In [ ]:
import numpy as np

def vectorize(docs, embedding_model=model_twitter, useSum=True):
    vectors = np.zeros((len(docs),50))
    for i in range(len(docs)):
        tokens = re.sub(regex, '', docs[i].lower()).split()
        embeddings = [embedding_model.get_vector(token) for token in tokens if token in embedding_model]
        if (len(embeddings) > 0):
            if (useSum):
                vectors[i] = sum(embeddings)
            else:
                vectors[i] = np.mean(embeddings, axis=0)
    return vectors

Let's now vectorize the training set
- and then print out one of the instances to check that the format is correct:

In [ ]:
train_x_vector = vectorize(train_x)
train_x_vector[:3].shape

Now that we have the vectorized training data, we can go ahead and train the model!

In [ ]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(max_iter=1000).fit(train_x_vector, train_y)
print(lr_model)

Let's test the model on the example tweets from the second tutorial:

In [ ]:
tweets = []
tweets.append('I can\'t believe how much fun I\'m having learning to train a text classifier using word embeddings!')
tweets.append('I am really confused. I want my mommy.')
tweets.append('The internet connection has been pretty annoying today!')
tweets.append('They just played my favourite song on the radio.')
tweets.append("I don't like going to the dentist.")
tweets.append("Can't wait for my three hour math class tomorrow morning. Yay!")


transformed_tweets = vectorize(tweets)
predictions = lr_model.predict(transformed_tweets)
predicted_probabilities = lr_model.predict_proba(transformed_tweets)

for i in range(len(tweets)):
  print(f'tweet:      {tweets[i]}')
  print(f'prediction: {predictions[i]}')
  print(f'confidence: {predicted_probabilities[i]}')
  print()

We can have a look at the coefficients of the logistic regression classification model.
- How many should there be?

In [ ]:
lr_model.coef_

Let's vectorize the validaton data and see how well the embeddings-based classifier performs on it:

In [ ]:
valid_x_vector = vectorize(valid_x)
pred_y = lr_model.predict(valid_x_vector)

from sklearn.metrics import accuracy_score

print(f'Accuracy: {accuracy_score(valid_y, pred_y)}')

import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_estimator(lr_model, valid_x_vector, valid_y, values_format='d')
plt.show()

So pretty much the same accuracy as the bag-of-words based classifier.
- but this time with feature vectors of size 50
- what happens to the accuracy if we:
  - use a different embedding: model_wiki instead of model_twitter
  - use a bigger (higher dimensional) embedding: download a larger one to see
  - use the average rather than the sum to generate the feature vector?
- try it out:

In [ ]:
# TODO

In [ ]:
# Vectorise
train_x_vector = vectorize(train_x, embedding_model=model_wiki)
# Fit model
lr_model = LogisticRegression(max_iter=1000).fit(train_x_vector, train_y)
# Evaluate results
valid_x_vector = vectorize(valid_x, embedding_model=model_wiki)
pred_y = lr_model.predict(valid_x_vector)

print(f'Accuracy: {accuracy_score(valid_y, pred_y)}')

ConfusionMatrixDisplay.from_estimator(lr_model, valid_x_vector, valid_y, values_format='d')
plt.show()

In [ ]:
# Vectorise
train_x_vector = vectorize(train_x, embedding_model=model_wiki, useSum=False)
# Fit model
lr_model = LogisticRegression(max_iter=1000).fit(train_x_vector, train_y)
# Evaluate results
valid_x_vector = vectorize(valid_x, embedding_model=model_wiki, useSum=False)
pred_y = lr_model.predict(valid_x_vector)

print(f'Accuracy: {accuracy_score(valid_y, pred_y)}')

ConfusionMatrixDisplay.from_estimator(lr_model, valid_x_vector, valid_y, values_format='d')
plt.show()

In [ ]:
# Vecorise
train_x_vector = vectorize(train_x, useSum=False)
# Fit model
lr_model = LogisticRegression(max_iter=1000).fit(train_x_vector, train_y)
# Evaluate results
valid_x_vector = vectorize(valid_x, useSum=False)
pred_y = lr_model.predict(valid_x_vector)

print(f'Accuracy: {accuracy_score(valid_y, pred_y)}')

ConfusionMatrixDisplay.from_estimator(lr_model, valid_x_vector, valid_y, values_format='d')
plt.show()

Also worth trying is doc2vec rather than word2vec embeddings:
- in theory they should give slightly better performance on sentence classification tasks
- see here: https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html

## Other uses for Word2Vec embeddings

Have a look at this inventive post that learns Word2Vec embeddings for automobiles described in CSV file:
- https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92
- The embeddings allow for similarity computations across  the vehicles.


In [ ]:
# TODO

## FastText (sub-word) embeddings

Try training a fasttext model:

https://pypi.org/project/fasttext/

You can have a look at the API and documentation at this [link](https://fasttext.cc).

Let's start by installing the fastText package.
(Note that, alterantively, Genism has a specific API for fastText that wraps the package we are going to use)

In [ ]:
!pip install fasttext

Now that we have installed the package, we can download one of the pre-trained models (you can find [here](https://fasttext.cc/docs/en/crawl-vectors.html) the list of available models)

In [ ]:
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip
!wget http://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
!gzip -d cc.en.300.bin.gz

--2025-03-12 10:16:10--  http://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.35.186.77, 13.35.186.32, 13.35.186.93, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.35.186.77|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘cc.en.300.bin.gz.3’

cc.en.300.bin.gz.3  100%[===================>]   4.19G  51.0MB/s    in 92s     

2025-03-12 10:17:42 (46.6 MB/s) - ‘cc.en.300.bin.gz.3’ saved [4503593528/4503593528]



Finally we can create a model istance:

In [ ]:
import fasttext

ft_model = fasttext.load_model('./cc.en.300.bin')

What is the size of the vocabulary?

In [ ]:
len(ft_model.get_words())

2000000

What is the size of the embeddings?

In [ ]:
ft_model.get_dimension()

300

How do we get the embedding of a word?

In [ ]:
ft_model.get_word_vector('car')

array([ 1.37498349e-01,  1.73652887e-01,  1.55173652e-02,  2.37601340e-01,
       -2.04669774e-01,  1.87128603e-01,  2.91468382e-01,  3.73304449e-03,
        4.89513204e-02,  2.60247648e-01, -8.31757560e-02, -1.56491295e-01,
        8.69184956e-02, -4.78181392e-02, -1.18615523e-01,  1.54884279e-01,
       -1.69153258e-01, -1.22394666e-01, -1.40723094e-01,  4.53762040e-02,
       -3.81604545e-02, -1.46799475e-01,  1.36863321e-01, -1.50539711e-01,
       -1.81618929e-02,  1.47275031e-02, -4.21417244e-02,  1.39445990e-01,
        4.28556725e-02,  3.05333138e-01, -1.66591443e-02,  2.91614503e-01,
       -3.32667604e-02, -4.68752086e-02,  2.67606732e-02,  8.71477500e-02,
       -5.38966954e-02,  1.33774459e-01,  8.50976482e-02, -6.40801638e-02,
       -1.03779674e-01, -1.81272849e-01, -1.10902652e-01,  1.66989714e-01,
       -1.99763715e-01,  4.53899801e-03, -1.89182535e-02, -2.78699417e-02,
       -2.93243110e-01, -5.81464916e-03,  1.75295904e-01,  1.08659819e-01,
       -1.88145787e-02,  

Alterantively you can use it as a Python dictionary

In [ ]:
ft_model['man']

array([ 2.05102742e-01, -1.17439777e-01, -1.55425537e-02,  1.79349303e-01,
       -2.28049219e-01, -1.24558017e-01,  1.23250902e-01,  5.38473055e-02,
        4.11795303e-02, -5.67295495e-03, -8.39752406e-02,  1.84653439e-02,
       -8.60689804e-02,  1.12930965e-02, -1.33117482e-01,  1.38373390e-01,
       -1.25412792e-01, -3.86665910e-02, -1.22192718e-01,  1.07491598e-01,
        7.44868889e-02,  1.43000856e-01, -2.68281717e-02,  7.85946473e-03,
       -3.62306088e-02, -2.08035931e-02,  9.95780062e-03, -5.96441068e-02,
        3.78727615e-02,  1.31439894e-01, -6.71993792e-02,  6.71701431e-02,
        1.11701377e-01, -1.82819627e-02, -1.02341630e-01,  9.74911451e-03,
        4.25241441e-02, -1.59083098e-01,  3.72390151e-02, -1.32961005e-01,
       -6.62351400e-02,  1.17778786e-01, -3.11643854e-02, -4.58469465e-02,
       -1.54904723e-01,  2.37882137e-03,  2.09096503e-02, -6.17710687e-03,
       -2.41204053e-01,  8.73834118e-02,  6.91391528e-02,  1.11958645e-01,
       -1.52415112e-02, -

Now we can test a bit the capabilities of fastText (examples are taken from [here](https://fasttext.cc/docs/en/unsupervised-tutorial.html)).

A base functionality is to search for close vectors in the embedding space

In [ ]:
word = 'asparagus'

ft_model.get_nearest_neighbors(word)

[(0.7746413350105286, 'aspargus'),
 (0.7368614077568054, 'Asparagus'),
 (0.7233701944351196, 'broccoli'),
 (0.7113494873046875, 'broccolini'),
 (0.7106742262840271, 'leeks'),
 (0.7096290588378906, 'artichokes'),
 (0.7028921842575073, 'asparagas'),
 (0.6759673953056335, 'kohlrabi'),
 (0.6757089495658875, 'radishes'),
 (0.6669561862945557, 'spinach')]

The first search takes additional time because the model is doing some sort of indexing in the embedding space. fastText uses a tool called FAISS to speed up the search in the embedding space (https://github.com/facebookresearch/faiss)

We can also solve analogies directly, without doing the computations by hand

In [ ]:
ft_model.get_analogies("man", "woman", "king")
# ft_model.get_analogies("psx", "sony", "nintendo")

[(0.6729540228843689, 'kings'),
 (0.6174856424331665, 'king-'),
 (0.6076122522354126, 'king.'),
 (0.5779222249984741, 'King'),
 (0.5720189213752747, 'king.The'),
 (0.5601366758346558, 'kingly'),
 (0.5598397254943848, 'prince'),
 (0.5453356504440308, 'king.But'),
 (0.5429127812385559, 'lord'),
 (0.5420817136764526, '-king')]

Note that we are dealing with a model using also subword n-grams, so we can get the representation and the similarity even of out of vocabulary words

For example we can search for the word **confuzzling** (which is an actual real fake word https://en.wiktionary.org/wiki/confuzzling):

In [ ]:
word = "confuzzling"
print(f'Is the word "{word}" in the fastText model vocabulary? {word in ft_model}')
for s, w in ft_model.get_nearest_neighbors(word):
    print(f'{s}: {w}')

Is the word "confuzzling" in the fastText model vocabulary? False
0.613641083240509: confuzzled
0.550931990146637: Confuzzled
0.4055270850658417: confuddled
0.3816613256931305: confusing
0.36328595876693726: confused
0.36032891273498535: nondescriptive
0.359388530254364: Wikipedia-Page-Suzannah-B-Troy-6-yrs-after-Misogynist-Cyber-Vandalism-Censorship-via-Deletion-on-a-page-about-Censorship-Wikipedia-Agrees-to-retur
0.3525550067424774: muddly
0.3524904251098633: official-like
0.3516576588153839: unflattering.Coordinating


This is why character n-grams are so important

We can also encode a sentence (sequence of tokens) into a single vector

In [ ]:
sent = ft_model.get_sentence_vector("Hello, is it me you're looking for? I can see it in your eyes. I can see it in your smile.")
sent

array([ 9.72567871e-03, -1.81693081e-02,  1.98467262e-02,  3.74992117e-02,
       -3.59138474e-02, -4.57771569e-02, -2.72532683e-02,  1.52418101e-02,
        3.52445617e-02, -7.53624178e-03,  9.12996754e-03, -2.10552160e-02,
        1.56272377e-03, -9.82039783e-05, -8.77684914e-03,  3.65382321e-02,
        3.64734158e-02, -7.05009839e-03, -2.33118087e-02,  6.08711829e-03,
       -1.99545734e-02,  1.54522238e-02,  6.07244065e-03, -8.34608357e-03,
       -3.15462463e-02, -9.89805069e-03, -5.55238919e-04, -6.88232016e-03,
       -1.15120355e-02,  1.13332726e-01,  2.06005611e-02, -1.42829418e-02,
       -4.10122750e-03, -1.72664113e-02,  3.63585819e-03, -1.34615647e-02,
       -4.80954209e-03,  4.18015793e-02, -8.03579669e-03, -5.55655640e-03,
       -3.50303599e-03,  5.23055904e-03, -2.29117647e-02,  2.92630084e-02,
        2.04842985e-02,  3.14234160e-02,  2.69671367e-03,  4.90053743e-02,
        1.12765245e-02,  4.62619215e-03,  5.81054529e-03, -1.93861313e-02,
        2.83782464e-03, -

(Maybe this can be useful to train a classifier for the twitter sentiment analysis...)

What if we want to train a model on our own (maybe with smaller embeddings)?
You can find a quick reference to the API [here](https://fasttext.cc/docs/en/python-module.html)

To train a model we need to convert the data set into the appropriate format, a text file containing al the samples we have.
We can start from the sentences we have from before

In [ ]:
with open('./data.txt', 'w') as f:
    f.write('\n'.join(sentences))

We can even choose whether to use a Skipgram based approach:

In [ ]:
ft_skip_model = fasttext.train_unsupervised('data.txt', model='skipgram', dim=30, minCount=5, ws=10)

Or a Continuos Bag-of-Words (CBoW) approach:

In [ ]:
ft_cbow_model = fasttext.train_unsupervised('data.txt', model='cbow')

Try to do the same visualisation we tried before, see how the same words are placed in a downprojection of the representation space.

In [ ]:
# TODO

### Building a word representation from the sub-words

How does fastText actually get to the word representation?
Can we get a look at how the representation is built?

If you want some additional details on what's going on under the hood, please take a look at this link: http://christopher5106.github.io/deep/learning/2020/04/02/fasttext_pretrained_embeddings_subword_word_representations.html

We can use the `get_subwords()` method to get all the character n-grams composing a word and the corresponding IDs.
The model we are using was trained with character 5-grams.

Let's see what happens when we pass a word to out function

In [ ]:
word = 'happiness'
sub_words, sub_word_ids = ft_model.get_subwords(word)
print(sub_words)
print(sub_word_ids)

['happiness', '<happ', 'happi', 'appin', 'ppine', 'pines', 'iness', 'ness>']
[   5303 3639000 2151425 3043427 3604793 2523952 2457215 2227900]


The first array contains the word to encode followed by its character n-grams, the second array contains the indices of the word and its n-grams in the fastText encoding matrix.

What happens if we pass a word that's not in the model?

In [ ]:
# TODO

Finally, we can compute the word representation as the average of the embeddings of the word itself and the embeddings of its character n-grams

In [ ]:
import numpy as np

word = 'happiness'
sub_words, sub_word_ids = ft_model.get_subwords(word)

embedding = np.mean([ft_model.get_input_vector(idx) for idx in sub_word_ids], axis=0)

embedding

array([ 2.43761260e-02,  3.08370143e-02, -1.82935037e-02,  1.92747042e-02,
        5.24661969e-03, -3.39045264e-02, -1.22417193e-02,  6.79974556e-02,
        2.97031924e-02,  1.41781345e-02, -5.99632552e-03,  3.24566700e-02,
       -6.19982481e-02,  7.10059237e-03,  3.57817323e-03, -3.29337753e-02,
        7.59602641e-04, -1.92011595e-02, -5.47431074e-02,  4.62299399e-03,
       -1.46642867e-02,  8.20630416e-02, -1.03985146e-02, -2.70312317e-02,
       -3.93761918e-02,  1.33094080e-02, -1.58767700e-02, -5.90743832e-02,
        4.97353896e-02,  8.92114043e-02,  5.08440509e-02, -3.04298364e-02,
       -3.62707898e-02,  1.76737197e-02, -1.84959592e-03, -2.82385275e-02,
        2.00545900e-02,  1.88715346e-02,  5.23229986e-02, -3.57023366e-02,
       -5.00665791e-02,  5.56163443e-03, -1.07403509e-02,  5.34310527e-02,
       -3.49775925e-02, -1.46959256e-03,  2.75344364e-02, -3.54247838e-02,
       -1.98210757e-02, -3.99537161e-02, -4.37548906e-02,  5.29136742e-03,
       -6.46033790e-03, -

Let's compare the embedding we retrieved with the original one

In [ ]:
np.sum((embedding - ft_model[word]) ** 2) ** 2

0.0